# Filtragem baseada em itens

 O uso de todas as recomendações de cada usuário para criar um conjunto de dados pode funcionar bem para sistemas pequenos. O que fizemos até agora é comparar um usuário com todos os outros e depois cada filme que cada usuário avaliou e um sistema com muitos produtos pode apresentar pouca semelhança entre usuário.

 A **filtragem baseada em itens** é utilizada em um conjunto de dados muito grandes, permite que os cálculos sejam feitos antecipadamente para retornar as recomendações mais rapidamente, **pré-computando itens similares**, usando média ponderada com itens similares. A comparação entre itens não mudarão com tanta frequência quanto comparações entre usuários. Os cálculos são feitos em baixo movimento ou em computadores separados.


In [1]:
import pandas as pd
import numpy as np

In [19]:
def distancia_euclidiana(x_i: np.array, y_i: np.array) -> float:
    return (sum((x_i - y_i)**2))**(0.5)

def similaridade(x_i: np.array, y_i: np.array) -> float:
    return 1/(1+ distancia_euclidiana(x_i, y_i))

def similaridade_df(item_1: str, item_2: str, coluna: str, base: pd.DataFrame) -> float:
    df = base.loc[base[coluna].isin([item_1, item_2])]
    df = df.dropna(axis=1)
    
    x_i = df.loc[df[coluna]==item_1].drop(coluna, axis=1).values[0]
    y_i = df.loc[df[coluna]==item_2].drop(coluna, axis=1).values[0]

    return similaridade(x_i, y_i)

def similaridade_por_item(item: str, coluna: str, base: pd.DataFrame, limite: float = 0.6) -> float:
    similaridades = {
            item_2: similaridade_df(item, item_2, coluna, base) 
            for item_2 in base[base[coluna]!=item][coluna].unique()
    }
    return {item: value for item, value in similaridades.items() if value >= limite}

In [12]:
filmes = {'Freddy x Jason': 
		{'Ana': 2.5, 
		 'Marcos': 3.0 ,
		 'Pedro': 2.5, 
		 'Adriano': 3.0, 
		 'Janaina': 3.0 },
	 
	 'O Ultimato Bourne': 
		{'Ana': 3.5, 
		 'Marcos': 3.5,
		 'Pedro': 3.0, 
		 'Claudia': 3.5, 
		 'Adriano': 4.0, 
		 'Janaina': 4.0,
		 'Leonardo': 4.5 },
				 
	 'Star Trek': 
		{'Ana': 3.0, 
		 'Marcos': 1.5,
		 'Claudia': 3.0, 
		 'Adriano': 2.0 },
	
	 'Exterminador do Futuro': 
		{'Ana': 3.5, 
		 'Marcos': 5.0 ,
		 'Pedro': 3.5, 
		 'Claudia': 4.0, 
		 'Adriano': 3.0, 
		 'Janaina': 5.0,
		 'Leonardo': 4.0},
				 
	 'Norbit': 
		{'Ana': 2.5, 
		 'Marcos': 3.0 ,
		 'Claudia': 2.5, 
		 'Adriano': 2.0, 
		 'Janaina': 3.5,
		 'Leonardo': 1.0},
				 
	 'Star Wars': 
		{'Ana': 3.0, 
		 'Marcos': 3.5,
		 'Pedro': 4.0, 
		 'Claudia': 4.5, 
		 'Adriano': 3.0, 
		 'Janaina': 3.0}
}

In [31]:
df_filmes = pd.DataFrame(filmes).T.reset_index().rename(columns={"index": "filme"})
print(df_filmes.shape)
df_filmes

(6, 8)


,filme,Ana,Marcos,Pedro,Adriano,Janaina,Claudia,Leonardo
0,Freddy x Jason,2.5,3.0,2.5,3.0,3.0,NaN,NaN
1,O Ultimato Bourne,3.5,3.5,3.0,4.0,4.0,3.5,4.5
2,Star Trek,3.0,1.5,NaN,2.0,NaN,3.0,NaN
3,Exterminador do Futuro,3.5,5.0,3.5,3.0,5.0,4.0,4.0
4,Norbit,2.5,3.0,NaN,2.0,3.5,2.5,1.0
5,Star Wars,3.0,3.5,4.0,3.0,3.0,4.5,NaN


Vamos recomendar um filme para o Leonardo

In [46]:
# Filmes que o Leonardo já viu

df_i = df_filmes[~df_filmes.Leonardo.isnull()][["filme", "Leonardo"]]
for filme_i in ['Freddy x Jason', 'Star Trek', 'Star Wars']:
    df_i[filme_i] = df_i.apply(lambda i: similaridade_df(i.filme, filme_i, "filme",  df_filmes), axis=1)
    df_i[f"Nota X {filme_i}"] = df_i[filme_i] * df_i["Leonardo"]
df_i

,filme,Leonardo,Freddy x Jason,Nota X Freddy x Jason,Star Trek,Nota X Star Trek,Star Wars,Nota X Star Wars
1,O Ultimato Bourne,4.5,0.348331,1.567492,0.255397,1.149286,0.326632,1.469842
3,Exterminador do Futuro,4.0,0.240253,0.961012,0.207992,0.831966,0.274292,1.097168
4,Norbit,1.0,0.472136,0.472136,0.376179,0.376179,0.294298,0.294298


Acima estamos calculando a similaridade entre os filmes que o Leonardo já assistiu com os filmes que o Leonardo ainda não assistiu.

In [49]:
print("Avaliação esperada para Freddy x Jason", sum(df_i["Nota X Freddy x Jason"])/sum(df_i["Freddy x Jason"]))
print("Avaliação esperada para Star Trek", sum(df_i["Nota X Star Trek"])/sum(df_i["Star Trek"]))
print("Avaliação esperada para Star Wars", sum(df_i["Nota X Star Wars"])/sum(df_i["Star Wars"]))

Avaliação esperada para Freddy x Jason 2.8288695093208878
Avaliação esperada para Star Trek 2.807912583423976
Avaliação esperada para Star Wars 3.196200842319029


In [50]:
similaridade_por_item("Star Trek", "filme", df_filmes, 0.0)

{'Freddy x Jason': 0.3483314773547883,
 'O Ultimato Bourne': 0.2553967929896867,
 'Exterminador do Futuro': 0.20799159651347807,
 'Norbit': 0.3761785115301142,
 'Star Wars': 0.2708131845707603}

In [56]:
def similaridade_por_coluna(coluna: str, base: pd.DataFrame, limite: float = 0.0) -> dict:
    results = {}
    for item in base[coluna].values:
        sim = similaridade_por_item(item, coluna, base, limite)
        results[item] = sim
    return results

In [57]:
similaridade_por_coluna("filme", df_filmes)

{'Freddy x Jason': {'O Ultimato Bourne': 0.3483314773547883,
  'Star Trek': 0.3483314773547883,
  'Exterminador do Futuro': 0.2402530733520421,
  'Norbit': 0.4721359549995794,
  'Star Wars': 0.3761785115301142},
 'O Ultimato Bourne': {'Freddy x Jason': 0.3483314773547883,
  'Star Trek': 0.2553967929896867,
  'Exterminador do Futuro': 0.3090169943749474,
  'Norbit': 0.1876127897984334,
  'Star Wars': 0.3266316347104093},
 'Star Trek': {'Freddy x Jason': 0.3483314773547883,
  'O Ultimato Bourne': 0.2553967929896867,
  'Exterminador do Futuro': 0.20799159651347807,
  'Norbit': 0.3761785115301142,
  'Star Wars': 0.2708131845707603},
 'Exterminador do Futuro': {'Freddy x Jason': 0.2402530733520421,
  'O Ultimato Bourne': 0.3090169943749474,
  'Star Trek': 0.20799159651347807,
  'Norbit': 0.18464218557642828,
  'Star Wars': 0.27429188517743175},
 'Norbit': {'Freddy x Jason': 0.4721359549995794,
  'O Ultimato Bourne': 0.1876127897984334,
  'Star Trek': 0.3761785115301142,
  'Exterminador do F

Assim esses valores ficam armazenados e não precisam ser carregados todas as vezes que precisamos fazer uma recomendação, basta consultar.

## Fazendo recomendações

In [65]:
def recomendar(base: pd.DataFrame, coluna: str, usuario: str) -> str:
    df_i = base[~base[usuario].isnull()][[coluna, usuario]]
    for item in base[base[usuario].isnull()][coluna].values:
        df_i[item] = df_i.apply(lambda i: similaridade_df(i[coluna], item, coluna,  base), axis=1)
        df_i[f"Nota X {item}"] = df_i[item] * df_i[usuario]
    notas_esperadas = {}
    for item in base[base[usuario].isnull()][coluna].values:
        notas_esperadas[item] = sum(df_i[f"Nota X {item}"])/sum(df_i[item])
    return notas_esperadas

In [69]:
recomendar(df_filmes, "filme", "Leonardo")

{'Freddy x Jason': 2.8288695093208878,
 'Star Trek': 2.807912583423976,
 'Star Wars': 3.196200842319029}

## Usuários x Itens

- Usuários:
    - Mais simples de implementar
    - Indicada para conjunto de dados menores (mantido integralmente em memória)
    - somento o cálculo de similaridade já pode ser útil
- Itens:
    - Mais rápida
    - Indicada para grandes conjuntos de dados
    - Necessita manter a tabela de similaridade de itens

**Desempenho**
- Itens tem desempenho melhor com dados esparços (pessoas deram notas para produtos diferentes, grupos)
- Usuários e itens tem depensempenho parecido para dados densos (pessoas deram notas para quase todos os filems)